In [1]:
%pylab inline
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import itertools,time
import sys, os
from collections import OrderedDict
from copy import deepcopy
import numpy as np
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import matplotlib.pyplot as plt
import itertools
import pickle

Populating the interactive namespace from numpy and matplotlib


/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
dataset_tr = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/train.txt.npy'
data_tr = np.load(dataset_tr)
dataset_te = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/test.txt.npy'
data_te = np.load(dataset_te)
vocab = '/afs/inf.ed.ac.uk/group/cup/data1/akash_itm\
/IPC_MAP/NTP/tensorflow_lda/lda-tf_new/data/20news_clean/vocab.pkl'
vocab = pickle.load(open(vocab,'r'))

In [3]:
vocab_size=len(vocab)
def onehot(data, min_length):
    return np.bincount(data, minlength=min_length)
data_tr = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_tr if np.sum(doc)!=0])
data_te = np.array([onehot(doc.astype('int'),vocab_size) for doc in data_te if np.sum(doc)!=0])
print data_tr.shape
print data_te.shape


(11258, 1995)
(7487, 1995)


In [4]:
n_samples_tr = 11258
n_samples_te = 7488
docs_tr = data_tr
docs_te = data_te

In [10]:
def xavier_init(fan_in, fan_out, constant=1): 
    """ Xavier initialization of network weights"""
    # https://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
    low = -constant*np.sqrt(6.0/(fan_in + fan_out)) 
    high = constant*np.sqrt(6.0/(fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), 
                             minval=low, maxval=high, 
                             dtype=tf.float32)
def log_dir_init(fan_in, fan_out,topics=50): 
    return tf.log((1.0/topics)*tf.ones([fan_in, fan_out]))

tf.reset_default_graph()
class VariationalAutoencoder(object):
    """ 
    See "Auto-Encoding Variational Bayes" by Kingma and Welling for more details.
    """
    def __init__(self, network_architecture, transfer_fct=tf.nn.softplus, 
                 learning_rate=0.001, batch_size=100):
        self.network_architecture = network_architecture
        self.transfer_fct = transfer_fct
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        print 'Learning Rate:', self.learning_rate
        
        # tf Graph input
        self.x = tf.placeholder(tf.float32, [None, network_architecture["n_input"]])
        self.keep_prob = tf.placeholder(tf.float32)
        self.is_training = tf.placeholder(tf.bool, [], name='is_training')
        self.samples = tf.placeholder(tf.int32)
        
        self.h_dim = float(network_architecture["n_z"])
        self.a0 = np.ones((1 , 5)).astype(np.float32)/5
        self.a1 = np.ones((1 , self.h_dim)).astype(np.float32)/self.h_dim
        self.a2 = np.ones((1 , 10)).astype(np.float32)/10
        
        self.mu2a0 = tf.constant((np.log(self.a0).T-np.mean(np.log(self.a0),1)).T)
        self.var2a0 = tf.constant(  ( ( (1.0/self.a0)*( 1 - (2.0/5.0) ) ).T +
                                ( 1.0/(25.0) )*np.sum(1.0/self.a0,1) ).T  )
        
        self.mu2a1 = tf.constant((np.log(self.a1).T-np.mean(np.log(self.a1),1)).T)
        self.var2a1 = tf.constant(  ( ( (1.0/self.a1)*( 1 - (2.0/self.h_dim) ) ).T +
                                ( 1.0/(self.h_dim*self.h_dim) )*np.sum(1.0/self.a1,1) ).T  )
        
        self.mu2a2 = tf.constant((np.log(self.a2).T-np.mean(np.log(self.a2),1)).T)
        self.var2a2 = tf.constant(  ( ( (1.0/self.a2)*( 1 - (2.0/10.0) ) ).T +
                                ( 1.0/(10*10.0) )*np.sum(1.0/self.a2,1) ).T  )
        
        
        
        # Create autoencoder network
        self._create_network()
        # Define loss function based variational upper-bound and 
        # corresponding optimizer
        self._create_loss_optimizer()
        
        # Initializing the tensor flow variables
        init = tf.initialize_all_variables()

        # Launch the session
        self.sess = tf.InteractiveSession()
        self.sess.run(init)
    
    def _create_network(self):
        n_z = self.network_architecture["n_z"]
        # Initialize autoencode network weights and biases
        self.network_weights = self._initialize_weights(**self.network_architecture)
        '''Adding 2 subtopics and 1 super topic.'''
        self.z_mean0,self.z_log_sigma_sq0 = \
            self._recognition_network(self.network_weights["weights_recog"], 
                                      self.network_weights["biases_recog"])
            
        self.z_mean_,self.z_log_sigma_sq_ = \
            self._recognition_network(self.network_weights["sub_01"], 
                                      self.network_weights["biases_sub_01"])
            
        self.z_mean1=self.z_mean_[:,0:10]
        self.z_mean2=self.z_mean_[:,10:2*10]
        self.z_mean3=self.z_mean_[:,2*10:3*10]
        self.z_mean4=self.z_mean_[:,3*10:4*10]
        self.z_mean5=self.z_mean_[:,4*10:5*10]
        
        self.z_log_sigma_sq1=self.z_log_sigma_sq_[:,0:10]
        self.z_log_sigma_sq2=self.z_log_sigma_sq_[:,10:2*10]
        self.z_log_sigma_sq3=self.z_log_sigma_sq_[:,2*10:3*10]
        self.z_log_sigma_sq4=self.z_log_sigma_sq_[:,3*10:4*10]
        self.z_log_sigma_sq5=self.z_log_sigma_sq_[:,4*10:5*10]
        
            
        self.z_mean__,self.z_log_sigma_sq__ = \
            self._recognition_network(self.network_weights["sub_02"], 
                                      self.network_weights["biases_sub_02"])
            
        self.z__mean1=self.z_mean__[:,0:n_z]
        self.z__mean2=self.z_mean__[:,n_z:2*n_z]
        self.z__mean3=self.z_mean__[:,2*n_z:3*n_z]
        self.z__mean4=self.z_mean__[:,3*n_z:4*n_z]
        self.z__mean5=self.z_mean__[:,4*n_z:5*n_z]
        self.z__mean6=self.z_mean__[:,5*n_z:6*n_z]
        self.z__mean7=self.z_mean__[:,6*n_z:7*n_z]
        self.z__mean8=self.z_mean__[:,7*n_z:8*n_z]
        self.z__mean9=self.z_mean__[:,8*n_z:9*n_z]
        self.z__mean10=self.z_mean__[:,9*n_z:10*n_z]
        
        self.z__log_sigma_sq1=self.z_log_sigma_sq__[:,0:n_z]
        self.z__log_sigma_sq2=self.z_log_sigma_sq__[:,n_z:2*n_z]
        self.z__log_sigma_sq3=self.z_log_sigma_sq__[:,2*n_z:3*n_z]
        self.z__log_sigma_sq4=self.z_log_sigma_sq__[:,3*n_z:4*n_z]
        self.z__log_sigma_sq5=self.z_log_sigma_sq__[:,4*n_z:5*n_z]
        self.z__log_sigma_sq6=self.z_log_sigma_sq__[:,5*n_z:6*n_z]
        self.z__log_sigma_sq7=self.z_log_sigma_sq__[:,6*n_z:7*n_z]
        self.z__log_sigma_sq8=self.z_log_sigma_sq__[:,7*n_z:8*n_z]
        self.z__log_sigma_sq9=self.z_log_sigma_sq__[:,8*n_z:9*n_z]
        self.z__log_sigma_sq10=self.z_log_sigma_sq__[:,9*n_z:10*n_z]
        


        # Draw one sample z from Gaussian distribution
        '''Adding another noise variable for super toipc: hard coding atm'''
        eps0 = tf.random_normal((self.samples, 5), 0, 1, 
                               dtype=tf.float32)
        
        eps1 = tf.random_normal((self.samples, 10), 0, 1, 
                               dtype=tf.float32)
        eps2 = tf.random_normal((self.samples, n_z), 0, 1, 
                               dtype=tf.float32)
        '''Adding RT for subtopics'''
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(self.z_mean0, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq0)), eps0))))
        self.z0 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z0,
                                                             is_training=self.is_training,updates_collections=None))
        
        self.sigma0 = tf.exp(self.z_log_sigma_sq0)
        
        self.z1 = tf.add(self.z_mean1, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq1)), eps1))
        self.z1 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z1,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma1 = tf.exp(self.z_log_sigma_sq1)
        
        self.z2 = tf.add(self.z_mean2, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq2)), eps1))
        self.z2 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z2,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma2 = tf.exp(self.z_log_sigma_sq2)
        
        self.z3 = tf.add(self.z_mean3, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq3)), eps1))
        self.z3 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z3,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma3 = tf.exp(self.z_log_sigma_sq3)
        
        self.z4 = tf.add(self.z_mean4, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq4)), eps1))
        self.z4 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z4,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma4 = tf.exp(self.z_log_sigma_sq4)
        
        self.z5 = tf.add(self.z_mean5, 
                        tf.mul(tf.sqrt(tf.exp(self.z_log_sigma_sq5)), eps1))
        self.z5 = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z5,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma5 = tf.exp(self.z_log_sigma_sq5)
        
        '''*****************************'''
        self.z1_ = tf.add(self.z__mean1, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq1)), eps2))
        self.z1_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z1_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma1_ = tf.exp(self.z__log_sigma_sq1)
        
        self.z2_ = tf.add(self.z__mean2, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq2)), eps2))
        self.z2_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z2_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma2_ = tf.exp(self.z__log_sigma_sq2)
        
        self.z3_ = tf.add(self.z__mean3, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq3)), eps2))
        self.z3_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z3_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma3_ = tf.exp(self.z__log_sigma_sq3)
        
        self.z4_ = tf.add(self.z__mean4, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq4)), eps2))
        self.z4_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z4_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma4_ = tf.exp(self.z__log_sigma_sq4)
        
        self.z5_ = tf.add(self.z__mean5, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq5)), eps2))
        self.z5_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z5_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma5_ = tf.exp(self.z__log_sigma_sq5)
        
        self.z6_ = tf.add(self.z__mean6, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq6)), eps2))
        self.z6_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z6_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma6_ = tf.exp(self.z__log_sigma_sq6)
        
        self.z7_ = tf.add(self.z__mean7, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq7)), eps2))
        self.z7_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z7_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma7_ = tf.exp(self.z__log_sigma_sq7)
        
        self.z8_ = tf.add(self.z__mean8, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq8)), eps2))
        self.z8_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z8_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma8_ = tf.exp(self.z__log_sigma_sq8)
        
        self.z9_ = tf.add(self.z__mean9, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq9)), eps2))
        self.z9_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z9_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma9_ = tf.exp(self.z__log_sigma_sq9)
        
        self.z10_ = tf.add(self.z__mean10, 
                        tf.mul(tf.sqrt(tf.exp(self.z__log_sigma_sq10)), eps2))
        self.z10_ = tf.nn.softmax(tf.contrib.layers.batch_norm(self.z10_,
                                                            is_training=self.is_training,updates_collections=None))
        
        self.sigma10_ = tf.exp(self.z__log_sigma_sq10)
        
        '''Adding subtopic reconstruction'''
            
#         self.theta = tf.matmul(self.z0,tf.concat(0,[self.z1, self.z2]))
        
        self.theta_ = tf.squeeze(tf.map_fn(lambda i: 
                                          tf.matmul(tf.expand_dims(tf.gather(self.z0,i),0),
                                                    tf.concat(0,[tf.expand_dims(tf.gather(self.z1,i),0),
                                                                tf.expand_dims(tf.gather(self.z2,i),0),
                                                                tf.expand_dims(tf.gather(self.z3,i),0),
                                                                tf.expand_dims(tf.gather(self.z4,i),0),
                                                                tf.expand_dims(tf.gather(self.z5,i),0)])),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
        
        self.theta = tf.squeeze(tf.map_fn(lambda i: 
                                          tf.matmul(tf.expand_dims(tf.gather(self.theta_,i),0),
                                                    tf.concat(0,[tf.expand_dims(tf.gather(self.z1_,i),0),
                                                                tf.expand_dims(tf.gather(self.z2_,i),0),
                                                                tf.expand_dims(tf.gather(self.z3_,i),0),
                                                                tf.expand_dims(tf.gather(self.z4_,i),0),
                                                                tf.expand_dims(tf.gather(self.z5_,i),0),
                                                                tf.expand_dims(tf.gather(self.z6_,i),0),
                                                                tf.expand_dims(tf.gather(self.z7_,i),0),
                                                                tf.expand_dims(tf.gather(self.z8_,i),0),
                                                                tf.expand_dims(tf.gather(self.z9_,i),0),
                                                                tf.expand_dims(tf.gather(self.z10_,i),0)])),
                                                   tf.constant(np.arange(self.batch_size)), dtype=tf.float32))
            
        self.x_reconstr_mean = \
            self._generator_network(tf.nn.dropout(self.theta, self.keep_prob),
                                    self.network_weights["weights_gener"],
                                    self.network_weights["biases_gener"])
            
    def _initialize_weights(self, n_hidden_recog_1, n_hidden_recog_2, 
                            n_hidden_gener_1,  
                            n_input, n_z):
        all_weights = dict()
        with tf.variable_scope("super"):
            all_weights['weights_recog'] = {
                'h1': tf.get_variable('h1',[n_input, 100]),
                'h2': tf.get_variable('h2',[100,100]),
                'out_mean': tf.get_variable('out_mean',[100, 5]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[100, 5])
            }
            all_weights['biases_recog'] = {
                'b1': tf.Variable(tf.zeros([100], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([100], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([5], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([5], dtype=tf.float32))
            }
        with tf.variable_scope("sub01"):
            all_weights['sub_01'] = {
                'h1': tf.get_variable('h1',[n_input, 100]),
                'h2': tf.get_variable('h2',[100,100]),
                'out_mean': tf.get_variable('out_mean',[100, 10*5]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[100, 10*5])}
            all_weights['biases_sub_01'] = {
                'b1': tf.Variable(tf.zeros([100], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([100], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([10*5], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([10*5], dtype=tf.float32))
            }
        with tf.variable_scope("sub02"):
            all_weights['sub_02'] = {
                'h1': tf.get_variable('h1',[n_input, n_hidden_recog_1]),
                'h2': tf.get_variable('h2',[n_hidden_recog_1, n_hidden_recog_2]),
                'out_mean': tf.get_variable('out_mean',[n_hidden_recog_2, n_z*10]),
                'out_log_sigma': tf.get_variable('out_log_sigma',[n_hidden_recog_2, n_z*10])
            }
            all_weights['biases_sub_02'] = {
                'b1': tf.Variable(tf.zeros([n_hidden_recog_1], dtype=tf.float32)),
                'b2': tf.Variable(tf.zeros([n_hidden_recog_2], dtype=tf.float32)),
                'out_mean': tf.Variable(tf.zeros([n_z*10], dtype=tf.float32)),
                'out_log_sigma': tf.Variable(tf.zeros([n_z*10], dtype=tf.float32))
            }
        with tf.variable_scope("gen"):
            all_weights['weights_gener'] = {
                'h2': tf.Variable(xavier_init(n_z, n_hidden_gener_1))
            }
            all_weights['biases_gener'] = {
                'b2': tf.Variable(tf.zeros([n_hidden_gener_1], dtype=tf.float32))
            }
        return all_weights
            
    def _recognition_network(self, weights, biases):
        # Generate probabilistic encoder (recognition network)
        layer_1 = self.transfer_fct(tf.add(tf.matmul(self.x, weights['h1']), 
                                           biases['b1'])) 
        layer_2 = self.transfer_fct(tf.add(tf.matmul(layer_1, weights['h2']), 
                                           biases['b2'])) 
        layer_do = tf.nn.dropout(layer_2, self.keep_prob)
        
        z_mean = tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_mean']),
                        biases['out_mean']),is_training=self.is_training,updates_collections=None)
        z_log_sigma_sq = \
            tf.contrib.layers.batch_norm(tf.add(tf.matmul(layer_do, weights['out_log_sigma']), 
                   biases['out_log_sigma']),is_training=self.is_training,updates_collections=None)     
        
        return (z_mean, z_log_sigma_sq)

    def _generator_network(self,z, weights, biases):
        # Generate probabilistic decoder (decoder network)
#         self.layer_do_0 = tf.nn.dropout(tf.nn.softmax(tf.contrib.layers.batch_norm(z)), self.keep_prob)
        self.layer_do_0 =z
        x_reconstr_mean = tf.add(tf.matmul(self.layer_do_0, 
                                           tf.nn.softmax(tf.contrib.layers.batch_norm(weights['h2'],\
is_training=self.is_training,updates_collections=None))),0.0)
#         x_reconstr_mean = tf.nn.softmax(tf.contrib.layers.batch_norm(tf.add(
#                     tf.matmul(self.layer_do_0, weights['h2']),0.0)))
    
        return x_reconstr_mean

            
    def _create_loss_optimizer(self):
        
        self.x_reconstr_mean+=1e-10
     
        reconstr_loss = \
            -tf.reduce_sum(self.x * tf.log(self.x_reconstr_mean),1)#/tf.reduce_sum(self.x,1) 

    
        latent_loss0 = 0.5*( tf.reduce_sum(tf.div(self.sigma0,self.var2a0),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a0 - self.z_mean0),self.var2a0),
                  (self.mu2a0 - self.z_mean0)),1) - 5 +\
                           tf.reduce_sum(tf.log(self.var2a0),1)  - tf.reduce_sum(self.z_log_sigma_sq0  ,1) ) 
        '''**************************************'''
        
        latent_loss_1 = 0.5*( tf.reduce_sum(tf.div(self.sigma1,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean1),self.var2a2),
                  (self.mu2a2 - self.z_mean1)),1) - 10 +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq1  ,1) ) 
        
        latent_loss_2 = 0.5*( tf.reduce_sum(tf.div(self.sigma2,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean2),self.var2a2),
                  (self.mu2a2 - self.z_mean2)),1) - 10 +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq2  ,1) ) 
        
        latent_loss_3 = 0.5*( tf.reduce_sum(tf.div(self.sigma3,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean3),self.var2a2),
                  (self.mu2a2 - self.z_mean3)),1) - 10 +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq3  ,1) ) 
        
        latent_loss_4 = 0.5*( tf.reduce_sum(tf.div(self.sigma4,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean4),self.var2a2),
                  (self.mu2a2 - self.z_mean4)),1) - 10 +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq4  ,1) ) 
        
        latent_loss_5 = 0.5*( tf.reduce_sum(tf.div(self.sigma5,self.var2a2),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a2 - self.z_mean5),self.var2a2),
                  (self.mu2a2 - self.z_mean5)),1) - 10 +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z_log_sigma_sq5  ,1) ) 
        
        '''**************************************'''
        
        latent_loss1 = 0.5*( tf.reduce_sum(tf.div(self.sigma1_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean1),self.var2a1),
                  (self.mu2a1 - self.z__mean1)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq1  ,1) )
        
        latent_loss2 = 0.5*( tf.reduce_sum(tf.div(self.sigma2_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean2),self.var2a1),
                  (self.mu2a1 - self.z__mean2)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a2),1)  - tf.reduce_sum(self.z__log_sigma_sq2  ,1) )
        
        latent_loss3 = 0.5*( tf.reduce_sum(tf.div(self.sigma3_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean3),self.var2a1),
                  (self.mu2a1 - self.z__mean3)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq3  ,1) )
        
        latent_loss4 = 0.5*( tf.reduce_sum(tf.div(self.sigma4_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean4),self.var2a1),
                  (self.mu2a1 - self.z__mean4)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq4  ,1) )
        
        latent_loss5 = 0.5*( tf.reduce_sum(tf.div(self.sigma5_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean5),self.var2a1),
                  (self.mu2a1 - self.z__mean5)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq5  ,1) )
        
        latent_loss6 = 0.5*( tf.reduce_sum(tf.div(self.sigma6_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean6),self.var2a1),
                  (self.mu2a1 - self.z__mean6)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq6  ,1) )
        
        latent_loss7 = 0.5*( tf.reduce_sum(tf.div(self.sigma7_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean7),self.var2a1),
                  (self.mu2a1 - self.z__mean7)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq7  ,1) )
        
        latent_loss8 = 0.5*( tf.reduce_sum(tf.div(self.sigma8_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean8),self.var2a1),
                  (self.mu2a1 - self.z__mean8)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq8  ,1) )
        
        latent_loss9 = 0.5*( tf.reduce_sum(tf.div(self.sigma9_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean9),self.var2a1),
                  (self.mu2a1 - self.z__mean9)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq9  ,1) )

        latent_loss10 = 0.5*( tf.reduce_sum(tf.div(self.sigma10_,self.var2a1),1)+\
        tf.reduce_sum( tf.mul(tf.div((self.mu2a1 - self.z__mean10),self.var2a1),
                  (self.mu2a1 - self.z__mean10)),1) - self.h_dim +\
                           tf.reduce_sum(tf.log(self.var2a1),1)  - tf.reduce_sum(self.z__log_sigma_sq10  ,1) )

        
        
#         latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq 
#                                            - tf.square(self.z_mean) 
#                                            - tf.exp(self.z_log_sigma_sq), 1)


        self.cost = tf.reduce_mean(reconstr_loss) \
    + tf.reduce_mean(latent_loss0 + latent_loss_1 + latent_loss_2 + latent_loss_3 + latent_loss_4 + latent_loss_5 +
                     latent_loss1 + latent_loss2 + 
                    latent_loss3 + latent_loss4 + latent_loss5 +
                    latent_loss6 + latent_loss7 + latent_loss8 +
                    latent_loss9 + latent_loss10) # average over batch
        
        # Use ADAM optimizer
        self.optimizer = \
            tf.train.AdamOptimizer(learning_rate=self.learning_rate,beta1=0.9).minimize(self.cost)
        
    def partial_fit(self, X):
        """Train model based on mini-batch of input data.
        
        Return cost of mini-batch.
        """
#         opt, cost,emb,kld = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2'],
#                                           self.kld),feed_dict={self.x: X,self.keep_prob: .9})
        opt,cost,emb = self.sess.run((self.optimizer, self.cost,self.network_weights['weights_gener']['h2']
                                      ),feed_dict={self.x: X,self.keep_prob: .7,self.is_training:True,self.samples:1})
        return cost,emb
    
    def test(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2,e3,e4 = self.sess.run((self.cost,self.z1,self.z2,self.z3_,self.z4_),
                                        feed_dict={self.x: np.expand_dims(X, axis=0),self.keep_prob: 1.0,
                                                   self.is_training:False,self.samples:1})
        return cost,e1,e2,e3,e4
    
    def batchtest(self, X):
        """Test the model and return the lowerbound on the log-likelihood.
        """
        cost,e1,e2,e3,e4 = self.sess.run((self.cost,self.z1,self.z2,self.z3_,self.z4_),
                                        feed_dict={self.x: X,self.keep_prob: 1.0,
                                                   self.is_training:False,self.samples:1})
        return cost,e1,e2,e3,e4


In [11]:
tf.reset_default_graph()
cost_plot=[]
test_batch = create_minibatch(docs_tr.astype('float32'))
def train(network_architecture, learning_rate=0.0001,
          batch_size=100, training_epochs=10, display_step=5):
    vae = VariationalAutoencoder(network_architecture, 
                                 learning_rate=learning_rate, 
                                 batch_size=batch_size)
    emb=0
    avg_kld=0
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        avg_kld=0.
        total_batch = int(n_samples_tr / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,emb = vae.partial_fit(batch_xs)
#             c,a,b,k,l = vae.batchtest(batch_xs)

#             sys.exit()
            # Compute average loss
            avg_cost += cost / n_samples_tr * batch_size
#             avg_kld += c / n_samples_tr * batch_size
            
            if np.isnan(avg_cost):
                print epoch,i,np.sum(batch_xs,1).astype(np.int),batch_xs.shape
                return vae,emb
#                 sys.exit()
        
        # Display logs per epoch step
        if epoch % display_step == 0:
            cost_plot.append(avg_cost)
            print "Epoch:", '%04d' % (epoch+1), \
                  "cost=", "{:.9f}".format(avg_cost)
#             print 'avg_cost = ',avg_kld
            test_c=0.
#             for i in range(int(n_samples_te / batch_size)):
#                 c,a,b,k,l = vae.batchtest(test_batch.next())
#                 test_c+=c / n_samples_te * batch_size
#             print 'avg_test_cost = ',test_c
    return vae,emb

In [12]:
tf.reset_default_graph()
network_architecture = \
    dict(n_hidden_recog_1=500, # 1st layer encoder neurons
         n_hidden_recog_2=500, # 2nd layer encoder neurons
         n_hidden_gener_1=1995, # 1st layer decoder neurons
         n_input=1995, # MNIST data input (img shape: 28*28)
         n_z=50)  # dimensionality of latent space

batch_size=200
learning_rate=0.002
def create_minibatch(data):
    rng = np.random.RandomState(10)
    
    while True:
        # Return random data samples of a size 'minibatch_size' at each iteration
        ixs = rng.randint(data.shape[0], size=batch_size)
        yield data[ixs]
        
minibatches = create_minibatch(docs_tr.astype('float32'))
vae,emb = train(network_architecture, training_epochs=300,batch_size=batch_size,learning_rate=learning_rate)
# plt.plot(cost_plot)


'''
Change log: decoder has no dropout or additional tranformations
'''

Learning Rate: 0.002
Epoch: 0001 cost= 1426.510490544
Epoch: 0006 cost= 1174.826095887
Epoch: 0011 cost= 1004.375392299
Epoch: 0016 cost= 902.918303451
Epoch: 0021 cost= 798.306628437
Epoch: 0026 cost= 715.587807481
Epoch: 0031 cost= 626.689940300
Epoch: 0036 cost= 633.446681730
Epoch: 0041 cost= 554.511035317
Epoch: 0046 cost= 571.864236712
Epoch: 0051 cost= 551.929433080
Epoch: 0056 cost= 571.197329230
Epoch: 0061 cost= 549.611122347
Epoch: 0066 cost= 530.981676810
Epoch: 0071 cost= 573.612386487
Epoch: 0076 cost= 536.001137472
Epoch: 0081 cost= 548.059886581
Epoch: 0086 cost= 537.226679821
Epoch: 0091 cost= 552.698500988
Epoch: 0096 cost= 555.043359141
Epoch: 0101 cost= 551.430238909
Epoch: 0106 cost= 568.198440180
Epoch: 0111 cost= 529.799530794
Epoch: 0116 cost= 540.474908706
Epoch: 0121 cost= 516.276202504
Epoch: 0126 cost= 544.059225094
Epoch: 0131 cost= 541.987688746
Epoch: 0136 cost= 533.606014499
Epoch: 0141 cost= 529.496638545
Epoch: 0146 cost= 538.424605090
Epoch: 0151 cost

/afs/inf.ed.ac.uk/group/cup/data1/akash_itm/IPC_MAP/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


'\nChange log: decoder has no dropout or additional tranformations\n'

In [13]:
def print_top_words(beta, feature_names, n_top_words=5):
    for i in range(len(beta)):
        print(" ".join([feature_names[j]
            for j in beta[i].argsort()[:-n_top_words - 1:-1]]))

print_top_words(emb, zip(*sorted(vocab.items(), key=lambda x: x[1]))[0])

game team player score play
god christian christ church sin
mb card slot scsi motherboard
ftp archive distribute available site
bike car ride honda road
angeles los san toronto detroit
journal index volume analysis remote
cost dollar cheaper pay larger
baseball game player team hitter
study energy medicine scientific experiment
israeli arabs arab israel territory
windows microsoft printer nt card
write article anyone gmt disclaimer
sexual homosexual male society marriage
waco fbi batf guilty trial
windows update directory unix utility
exception panel neutral wiring permit
team game rangers playoff leafs
organization participate former california association
flight station space solar spacecraft
signal battery hole amp cycle
key chip algorithm encrypt security
gun crime criminal weapon handgun
sale please thanks shipping sell
phone escrow chip clipper warrant
unix windows font utility graphic
president secretary meet february morning
engineer sky pat flight station
security encryption a

In [14]:

cost=[]
e1=[]
e2=[]
e3=[]
e4=[]
idx=0
for doc in docs_te:
    idx+=1
    doc = doc.astype('float32')
    n_d = np.sum(doc)
    c,a,b,k,l=vae.test(doc)
    cost.append(c)
#     print c,n_d
    e1.append(a)
    e2.append(b)
    e3.append(k)
    e4.append(l)
    if idx%1000==0:
        print idx
    
print ((np.mean(np.array(cost))))
# print np.mean(e1,0)
# print np.mean(e2,0)

1000
2000
3000
4000
5000
6000
7000
1855.25


In [15]:
print np.argsort(np.mean(e1,0)[0])+1
print np.argsort(np.mean(e2,0)[0])+1
print np.argsort(np.mean(e3,0)[0])+1
print np.argsort(np.mean(e4,0)[0])+1
print np.mean(e1,0)#[0]
print np.mean(e2,0)#[0]
print np.sort(np.mean(e3,0))#[0]
print np.sort(np.mean(e4,0))#[0]
print np.exp(np.mean(np.array(cost)))

[ 9  7  3  2 10  8  1  6  4  5]
[ 2  3  7  8  1 10  6  5  9  4]
[16 38 33  5 32 43 40 10 24 45 18 41 48 21 20  6  9  3 31 19  1 36 30 13 17
 44 12 39 34  2 23 49 42  4 50 15 35 28 11  7  8 46 47 22 37 25 29 14 27 26]
[40  4 12 46 26  7 45  3 24 17 22 33 27 16 31  6 47 32 36 29 34 19 20 50 48
 10 25 44 38 23 41 28 18  1 13 11 30 42  2 15  9 43 14 37  5  8 21 49 35 39]
[[ 0.04138938  0.03004234  0.02523973  0.36969057  0.37911797  0.04435895
   0.02345861  0.03943225  0.01490024  0.03236888]]
[[ 0.10076236  0.06011886  0.06460684  0.14951421  0.11817452  0.11710971
   0.07608031  0.08452383  0.12110629  0.10800172]]
[[ 0.00269156  0.00622311  0.00800942  0.01071974  0.01122875  0.01301569
   0.0137848   0.01427796  0.01432504  0.01464061  0.01491903  0.01567364
   0.01641777  0.01683908  0.01769745  0.01800199  0.01804366  0.01811566
   0.01863624  0.01871497  0.01884321  0.0191149   0.01932066  0.01995368
   0.02016581  0.02019441  0.02081196  0.02085668  0.02089837  0.02132988
   0.022

In [19]:
pickle.dump(cost,open('cost','w'))
pickle.dump(e1,open('e1','w'))
pickle.dump(e2,open('e2','w'))

In [19]:
e=e2
print np.amax(np.mean(e,0))
print np.min(np.mean(e,0))

0.066614
0.00266949


In [98]:
test_batch = create_minibatch(docs_te.astype('float32'))
def test(vae,minibatches):
    emb=0
    avg_kld=0
    print batch_size
    # Training cycle
    for epoch in range(10):
        avg_cost = 0.
        avg_kld = 0.
        total_batch = int(n_samples_te / batch_size)
        print total_batch
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = minibatches.next()
            # Fit training using batch data
            cost,a,b = vae.batchtest(batch_xs)
            cost=cost
            # Compute average loss
            avg_cost += cost / total_batch#n_samples_tr * batch_size


        # Display logs per epoch step
#         if epoch % display_step == 0:
        cost_plot.append(avg_cost)
        print "Epoch:", '%04d' % (epoch+1), \
              "cost=", "{:.9f}".format(np.exp((avg_cost/90)))
        print 'avg_cost = ',avg_cost

test(vae,test_batch)

200
37
Epoch: 0001 cost= 1951.299849720
avg_cost =  681.862591718
37
Epoch: 0002 cost= 1802.893371419
avg_cost =  674.743327373
37
Epoch: 0003 cost= 2080.210038171
avg_cost =  687.620173274
37
Epoch: 0004 cost= 2131.480287028
avg_cost =  689.811478898
37
Epoch: 0005 cost= 1697.812007509
avg_cost =  669.3386082
37
Epoch: 0006 cost= 1812.600706940
avg_cost =  675.226615287
37
Epoch: 0007 cost= 2147.505452401
avg_cost =  690.485597353
37
Epoch: 0008 cost= 1754.365650309
avg_cost =  672.287635597
37
Epoch: 0009 cost= 1986.632824159
avg_cost =  683.477679278
37
Epoch: 0010 cost= 1301.936128341
avg_cost =  645.444698849


In [65]:
print np.mean(np.sum(docs_tr.astype('float32'),1))

88.0701


In [ ]:
[2 1 0 7 3 8 4 5 6 9]
thanks hus advance appreciate modem
article write like get one
god people bible faith one
team rangers game player leafs
people population arab village government
god people christian truth evidence
thanks motherboard sale hd ide
virtual software interface graphic application
government arab israeli turks population
spacecraft encryption key km flight

[4 1 9 2 8 6 5 0 3 7]
thanks motherboard sale hd ide
god people christian truth evidence
virtual software interface graphic application
god people bible faith one
article write like get one
people population arab village government
spacecraft encryption key km flight
thanks hus advance appreciate modem
government arab israeli turks population
team rangers game player leafs